[Reference](https://www.kaggle.com/code/benihime91/pytorch-fasterrcnn)

### Imports

In [1]:
import os
import numpy as np
import pandas as pd
from PIL import Image

# Sk learn
from sklearn.model_selection import train_test_split

# PyTorch
import torch
from torch.utils.data import Dataset, DataLoader
import torchvision
from torchvision import transforms, datasets, models
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
from torchvision.models.detection.mask_rcnn import MaskRCNNPredictor


In [3]:
# Count the total number of images

img_count = list(sorted(os.listdir('../data/mask_detection/images/')))
len(img_count)

853

#### Split the data

In [9]:
train_img, test_img = train_test_split(img_count, test_size = 0.2)
print(f"No. of training images: {len(train_img)}")
print(f"No. of test images: {len(test_img)}")

No. of training images: 682
No. of test images: 171


### DataSet Class

In [ ]:
class MaskDataset(Dataset):
    def __init__(self, img_dir, annot_dir, img_list, transforms):
        self.transforms = transforms
        self.imgs = img_list
        self.img_dir = img_dir
        self.annot_dir = annot_dir

    def __len__(self):
        return len(self.imgs)

    def __getitem__(self, idx):
        f_image = 'maksssksksss'+ str(idx) + '.png'
        f_label = 'maksssksksss'+ str(idx) + '.xml'
        
        img_path = os.path.join(self.img_dir, f_image)
        label_path = os.path.join(self.annot_dir, f_label)
        
        # Open the image
        img = Image.open(img_path).convert("RGB")
        
        # target = 
        

### DataLoader Class